In [2]:
#PASO 1

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
#PASO 2

df = pd.read_csv(r"C:\Users\marco\OneDrive\Documentos\GitHub\WhiteHosting\Marco\AsturiasDF_limpio.csv")
lista_ids = df["ID"]

In [6]:
#PASO 3

lista_coordenadas=[]
opts= Options()
opts.add_argument ("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
opts.add_argument("--headless")
#opts.add_argument('--blink-settings=imagesEnabled=false')    /// Desactivado porque da error si no las coordenadas
driver= webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)
for x in lista_ids:
    url_hab_ago= f"https://www.airbnb.es/rooms/{x}?source_impression_id=p3_1713604628_Lgj1zTSBQ4yrgaUA&check_in=2024-08-12&guests=1&adults=1&check_out=2024-08-18"
    driver.get(url_hab_ago)
    sleep (2)
    try:
        driver.find_element("xpath", '//*[@aria-label="Cerrar"]').click()
    except:
        pass
    try:
        mapa = driver.find_element("xpath", '//*[@data-section-id="LOCATION_DEFAULT"]')
        driver.execute_script("arguments[0].scrollIntoView(true);", mapa)
    except:
        pass
    sleep (1)
    html=driver.page_source
    soup_ago= bs(html, "lxml")

    #COORDENADAS
    coord0= soup_ago.find("div", {"style":"margin: 0px 5px; z-index: 1000000; position: absolute; left: 0px; bottom: 0px;"})
    if coord0 is not None:
        coord1= str(coord0)
        index= coord1.find("maps?ll")
        index2= coord1.find("&amp")
        lista_coordenadas.append(coord1[index+8:index2])
    else:
        lista_coordenadas.append(None)

driver.close()

In [7]:
#4 PASO

df["Localización"] = lista_coordenadas

In [8]:
#5 PASO

df.to_csv("AsturiasDF_limpio_con_coordenadas.csv", index=False)